# Table des Matières
* [Nombres](#nombres)
   - [Nombres aléatoires](#aléatoires)
   - [Précision des calculs](#précision)
* [Fonctions](#functions)
   - [Gradual Typing](#gradual)
   - [Subsets](#subsets)
   - [Multi-Methods](#methods)
* [Concurrency](#concurrency)
   - [Parallel Processing](#parallel)
   - [Hyper Operators](#hyperoperators)
   - [Hyper Sequences](#hyperseqs)
   - [Asynchrone](#promises)
* [Grammars](#grammars)   


<a name="nombres"> </a>
# Nombres

<a name="aléatoires"> </a>
## Nombres aléatoires

Ceci est un problème généralisé, depuis toujours. Est-ce que votre langage de programmation as un mot clé `rand`, qui donne une réponse entre 0 et presque 1, ou une fonction `rand()`, qui donne une réponse entre 1 et l'argument?  Est-ce que c'est en point flottant? Est-ce qu'il faut l'arrondir? le tronquer? 


Raku fournit une fonction `roll` qui sélectionne au hasard un élément d'une liste (plutôt que de générer un entier)

In [36]:
(1000..9999).roll;

1561

Un exemple du genre d'utilisation: voici un dungeon master qui veut donner des attributs au hasrd à un joueur: 

In [48]:
my @attributes = <Str Int Dex Con Cha>;

my @d6 = 1..6;

for @attributes -> $atr {
    my $stat = @d6.roll(3).sum;

    say "$atr: $stat"; 
}

Str: 14
Int: 8
Dex: 11
Con: 9
Cha: 10


Mais peut-être un dungeon master plus généreux prendrait les meilleurs 3 dés sur 5:

In [47]:
my @attributes = <Str Int Dex Con Cha>;

my @d6 = 1..6;

for @attributes -> $atr {
    my $stat = @d6
        .roll(5)
        .sort(-*)
        .head(3).sum;
        
    say "$atr: $stat"; 
}

Str: 17
Int: 12
Dex: 10
Con: 16
Cha: 15


Enfin, `roll` permet la duplication, fait la sélection avec des doubles. Mais si ont veut avoir une sélection sans doubles? 

In [57]:
my @friends = <Bob Martial Jean Freddie Bob Simon Jean>;
say @friends.pick(3);

(Bob Bob Jean)


<a name="précision"></a> 
## Précision des calculs

Le point-flottant est un problème depuis toujours. Sur STack Overflow, il y a d'innombrables questions dur le poinmt flottant. 

En mathèmathiques, l'énoncé `0.3 = 0.1 + 0.2`  est vrai. Mais, en python? en ruby?

In [77]:
!ruby 'puts 0.3 == 0.1 + 0.2 ? "Yes" : "no" '

Undeclared routine:
    ruby used at line 2


Python, Perl5, C, et maintes autres langages donnent la même reponse. 

Mais, en Raku, la réponse est correcte!

In [79]:
say 0.1 + 0.2 == 0.3 ?? "yes" !! "No" ;

yes


In [81]:
say 0.3.nude;

(3 10)


In [82]:
say (1/12 + 1/5).nude;

(17 60)


In [83]:
0.3.WHAT

(Rat)

Dans Haskell, ce sont des "Rationals". Dans Raku, ce sont des Rat. (Et si les chiffres deviennent assez frands, des BigRat)

C'est plus lent que du point-flottant `hardware`, mais ça donne la bonne réponse, ce que le point flottant ne fait pas. 

Un autre exemple: 7\over 3

In [86]:
say -7/3;

-2.333333


Dans d'autres langages, on a les résultats suivants: 



<a name="functions"></a> 
# Fonctions

Des fonctions, methodes ou lambdas en raku sont des appelés des`subroutines`. Il y a maintes façons de les déclarer:

In [124]:
sub my-func { say "Look ma, no args!" }
my-func;

Look ma, no args!


In [ ]:
On peut les placer dans n'importe quel contenant (ou pas)

In [12]:
my &c = sub { say "Look ma, no name!" }
c;     # OUTPUT: «Look ma, no name!␤» 
 
my Any:D $f = sub { say 'Still nameless...' }
$f();  # OUTPUT: «Still nameless...␤» 

my $nameless = -> { say 'also nameless' } # un `pointy block`
$nameless();
 
my Code \a = sub { say ‚raw containers don't implement postcircumfix:<( )>‘ };
a.();  # OUTPUT: «raw containers don't implement postcircumfix:<( )>␤»

Look ma, no name!
Still nameless...
also nameless
raw containers don't implement postcircumfix:<( )>


Une fonction peut recevoir des arguments, séparés par des virgules, même avec des valeurs par défault:

In [125]:
sub exclaim ( $phrase, $name = 'John' ) {
    say "$phrase $name !!!!";
}

exclaim( 'Salut, mon chum' );

Salut, mon chum John !!!!


<a name="gradual"></a>
## Gradual Typing

`Gradual typing` veut dire qu'on n'a pas besoin de spécifier des types d'arguments pour une fonction, mais si on les spécifie, ça permet de mieux gérer la compilation et l'exécution. Raku amène ça un peu plus loin que d'autres. 

L'ensemble d'arguments ainsi que la valeur de retour est la signature d'une fonction. 

In [126]:
sub fmt( $lineno, Str $str ) {
    say "$lineno: $str";
}
    
fmt( 1, "This must be a string" );
#fmt 2, [ 1, 2, 3];
#fmt 3, π;
say pi == π;
say pi;


1: This must be a string
True
3.141592653589793


On peut aussi mettre des contraintes:

In [123]:
    sub fmt( $lineno where * < 10, Str $str where *.chars == 5 ) {
        say "$lineno, $str";
    }
    
    #fmt( 1, "This must be a string" );
    fmt( 1, "afive");
    fmt(12,"afive");

1, afive


Constraint type check failed in binding to parameter '$lineno'; expected anonymous constraint to be met but got Int (12)

<a name="subsets"></a>
## Subsets

Une autre façon de faire ce genre de chose est de déclarer des types avec contraintes:

In [121]:
subset SmallStr of Str where *.chars < 10;

Redeclaration of symbol 'SmallStr'

In [159]:
sub baby-twitter( SmallStr $str ) {
   say "This string must be less that 10 chars: $str"
}

#baby-twitter 'a longer string';
baby-twitter 'tiny twit';

This string must be less that 10 chars: tiny twit


<a name="methods"></a>
## Multi-Methods

Plusieurs langages de programmation oint la capacité de définir plusieurs fois la même fonction ou methode, distingué par des arguments différents.
Raku amène ça à un autre niveau.

In [169]:
subset Prime      of Int   where *.is-prime;
subset BigPrime   of Prime where * >  10_000;
subset SmallPrime of Prime where * <= 10_000;

Redeclaration of symbol 'Prime'

In [167]:
multi sub test-prime ( BigPrime   $num ) { say "Prime number! Nice and big"; }
multi sub test-prime ( SmallPrime $num ) { say "Puny prime number";          }
multi sub test-prime (            $num ) { say "Gimme primes!";              }

&test-prime

In [168]:
test-prime 3;
test-prime(31337);
test-prime 100_000;

Puny prime number
Prime number! Nice and big


Ambiguous call to 'test-prime(Int)'; these signatures all match:
:($num)
:($num)
:($num)

On peut même étendre des classes:

In [171]:
class Numbers {
    multi method id ( Numeric $num ) { say "$num is a number"       }
    multi method id (         $num ) { say "$num is something else" }
}
 
class SmarterNumbers is Numbers {
    multi method id ( Numeric $num where * == π ) { say "Mmmm yummy pie!" }
}
 
SmarterNumbers.new.id: 42;
SmarterNumbers.new.id: π;
SmarterNumbers.new.id: 'blah';

42 is a number
Mmmm yummy pie!
blah is something else


<a name=concurrency></a>
# Concurrency

<a name=parallel></a>
## Parallel Processing

Le traitement en parallel est souvent difficile à gérer, et très difficile à debugger. En fait, sans passer par des processus, la majorité des langages de scripting communs aujourd'hui ont un 'Global Interpreter Lock' qui fait en sorte que on ne peut pas rouler en multi-thread. Il faut alors se pencher sur des processus séparés, ce qui rends les choses plus difficiles encore. 

<a name="hyperoperators"></a>
## Hyper Operators

Imaginons que les deux vecteurs `@row1` et `@row2` ont des millions d'éléments. 

In [137]:
my @row1 = 5, 20, 3;
my @row2 = 3, 2, 1;

# say row_add( @row1, @row2 );

[3 2 1]

Un premier essai très simple: 

In [128]:
sub row_add( @a, @b ) {
    die if @a.elems != @b.elems;
    
    my @result;
    for 0 .. @a.end -> $i {
        push @result, @a[$i] + @b[$i];
        }
     return @result;   
}

&row_add

C'est sériel, un pas après l'autre. 

In [145]:
sub row_add( @a, @b ) {
    die if @a.elems != @b.elems;
    
    return map { @a[$_] + @b[$_] }, 0 .. @a.end;
}

&row_add

La fonction `keys` fournit la liste d'indexes: 

In [155]:
sub row_add( @a, @b ) {
    die if @a.elems != @b.elems;
    
    return map { @a[$_] + @b[$_] }, @a.keys;
}

&row_add

L'opérateur Zip (semblable à la fonction zip en python):

In [134]:
sub row_add( @a, @b ) {
    die if @2.elems != @b.elems;

    return @a Z+ @b;
}

&row_add

Mais toutes ces méthodes sont en série. 

In [156]:
sub row_add( @a, @b ) {
    return @a »+« @b; # les deux longeurs doivent être identiques (exception si pas égal)
    
    #return @a <<+<< @b; retourne resultat de taille de @b, @a est réutilisé
    #return @a >>+>> @b; retourne resultat de taille de @a, @b est réutilisé
    #return @a <<+>> @b; retourne un résultat de taille du plus grand vecteur, le plus court est réutilisé
    
}

&row_add

In [138]:
say row_add( @row1, @row2 );

[8 22 4]


Les  `hyper-operators` sont potentiellement multi-thread; ils ne forcent pas le multi-thread, mais disent au compilateur que il n'y a pas d'effets secondaires, donc il peut y aller si c'est possible.

Un exemple très utile c'est quand on fait du `scaling`:

In [84]:
sub scale ( @a, $n ) { return @a »*» $n }

&scale

Une différence importante entre le meta-operateur `Z` et les hyper-opérateurs « et » est que (autres que le `multi-threading` est que `Z` ne fait qu'un niveau, tandis que « et » traversent complètement la structure. 

<a name="hyperseqs"></a>
## Hyper Sequences

In [177]:
for (1..4).race( batch => 1 ) {
    say "Doing $_";
    sleep 1;
}
say "Code took {now - INIT now} seconds to run";

Doing 1
Doing 2
Doing 3
Doing 4
Code took 4.01894736 seconds to run


la methode `race` ne préserve pas l'ordre, `hyper` est identique mais préserve l'ordre. 

<a name="promises"></a>
## Asynchrone (i.e. Promises)

Expression asynchrone avec `start`/`await`:

In [179]:
my @promises = ^3 .map: {
    start {
        .say; sleep 1;
        $_ * 4;
    }
};
say "Commencé! {now - INIT now}";
say await @promises;
say "Complet! {now - INIT now}";

0
2
1
Commencé! 0.0004653
(0 4 8)
Complet! 1.00308921


Une autre variété via la class `Promise`:

In [182]:
Promise.in(5).then: -> $v { say "Ça fait {now - INIT now} secondes!" };
sleep 7;
say "Durée total {now - INIT now} secondes"

Ça fait 5.00524502 secondes!
Durée total 7.00046612 secondes


<a name="grammars"></a>
# Grammars

Perl était reconnu pour ses regexs. MUltiples langages de programmation on copiés ou directement implantés les regexs de perl, même s'ils ne servaient pas autant. Mais les regexs sont difficile à comprendre, même très difficile à écrire moindrement que les textes deviennent compliqués. 

Prenons, par exemple, une date iso, em perl5 on ferait:

mais, si on a besoin de lire deux dates, on fait quoi?  Prenons un fichier avec des lignes tel que: `IsoDate IsoDate Titre`

In [175]:
grammar LogLine {
    token year  { <.digit> ** 4 <?{ $/.Num >= 1900  }> }
    token month { <.digit> ** 2 <?{ $/.Num ~~ 1..12 }> }
    token day   { <.digit> ** 2 <?{ $/.Num ~~ 1..31 }> }
    
    token ISO-Date { <year> '-' <month> '-' <day> }
    regex book-name { \S .*? }
    token start-date { <ISO-Date> }
    token stop-date  { <ISO-Date> }
    rule TOP { ^ <start-date> <stop-date> <book-name> $ }
    }

Redeclaration of symbol 'LogLine3'

In [174]:
my $line = ' 2010-02-28 2015-03-14 Harry Potter and the Methods of Rationality';

LogLine.parse($line) or die;

say $/<book-name>.Str.perl;

"Harry Potter and the Methods of Rationality"


`Grammar` crée un `recursive-descent` parser. Pour faciliter les choses, il y a deux outils `Grammar::Tracer` et `Grammar::Debugger` pour aider à vérfier le code. 

### Palindromes

In [76]:
say (^100).grep({ $_ eq $_.flip }).elems;

19
